In [33]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from apscheduler.schedulers.background import BackgroundScheduler
import plotly.express as px
import pandas as pd
import mysql.connector

# Initialize the Dash app with suppress_callback_exceptions
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)
server = app.server

def fetch_data():
    try:
        cnx = mysql.connector.connect(
            user='root',
            password='',
            host='localhost',
            database='brothers_analysis'
        )
        # Retrieve data from the necessary tables
        opd_query = "SELECT * FROM opd"
        sales_query = "SELECT * FROM sales"
        purchase_query = "SELECT * FROM purchase"
        lab_query = "SELECT * FROM lab"

        opd_df = pd.read_sql(opd_query, cnx)
        sales_df = pd.read_sql(sales_query, cnx)
        purchase_df = pd.read_sql(purchase_query, cnx)
        lab_df = pd.read_sql(lab_query, cnx)
        
        cnx.close()
    
        return opd_df, sales_df, purchase_df, lab_df
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

# Define a function to update the data periodically
def update_data():
    global opd_df, sales_df, purchase_df, lab_df
    opd_df, sales_df, purchase_df, lab_df = fetch_data()

# Schedule the data update job to run every hour
scheduler = BackgroundScheduler()
scheduler.add_job(update_data, 'interval', minutes=5)
scheduler.start()

# Initial data fetch
opd_df, sales_df, purchase_df, lab_df = fetch_data()

# Preprocess OPD data
opd_df['doctor_name'].fillna('Not Specified', inplace=True)
opd_df['visit_type'].fillna('Not Specified', inplace=True)
opd_df['age'].fillna('Not Specified', inplace=True)

# Preprocess Sales and Purchase data
sales_df.fillna('Not Specified', inplace=True)
purchase_df.fillna('Not Specified', inplace=True)

# Preprocess Lab data
lab_df['test_name'].fillna('Not Specified', inplace=True)
lab_df['time'].fillna('Not Specified', inplace=True)

# Define the app layout with navigation buttons
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    dbc.NavbarSimple(
        children=[
            dbc.NavItem(dbc.NavLink("OPD Dashboard", href="/opd")),
            dbc.NavItem(dbc.NavLink("Sales Dashboard", href="/sales")),
            dbc.NavItem(dbc.NavLink("Lab Dashboard", href="/lab")),
        ],
        brand="Dashboard Home",
        brand_href="/opd",
        color="primary",
        dark=True,
    ),
    html.Div(id='page-content')
])

# Define the OPD dashboard layout
opd_layout = dbc.Container(fluid=True, children=[
    dbc.Row([
        dbc.Col([
            html.H2("OPD Management Analysis"),
            html.Hr(),
            dbc.Row([
                dbc.Col([
                    html.Label("Doctor Name", className="dropdown-container"),
                    dcc.Dropdown(
                        id='doctor-name-filter',
                        options=[{'label': name, 'value': name} for name in opd_df['doctor_name'].unique()],
                        multi=True,
                        placeholder="Select Doctor(s)"
                    ),
                ], width=4),
                dbc.Col([
                    html.Label("Visit Type", className="dropdown-container"),
                    dcc.Dropdown(
                        id='visit-type-filter',
                        options=[{'label': visit, 'value': visit} for visit in opd_df['visit_type'].unique()],
                        multi=True,
                        placeholder="Select Visit Type(s)"
                    ),
                ], width=4),
                dbc.Col([
                    html.Label("Age Group", className="dropdown-container"),
                    dcc.Dropdown(
                        id='age-group-filter',
                        options=[{'label': age, 'value': age} for age in opd_df['age'].unique()],
                        multi=True,
                        placeholder="Select Age Group(s)"
                    ),
                ], width=4),
            ]),
        ], width=12),
        dbc.Col([
            dbc.Row([
                dbc.Col(dcc.Graph(id='visit-type-chart', config={'displayModeBar': False}, className="graph-container"), width=4),
                dbc.Col(dcc.Graph(id='doctor-demand-chart', config={'displayModeBar': False}, className="graph-container"), width=8),
            ]),
            dbc.Row([
                dbc.Col(dcc.Graph(id='registration-traffic-chart', config={'displayModeBar': False}, className="graph-container"), width=4),
                dbc.Col(dcc.Graph(id='gender-analysis-chart', config={'displayModeBar': False}, className="graph-container"), width=4),
                dbc.Col(dcc.Graph(id='age-analysis-chart', config={'displayModeBar': False}, className="graph-container"), width=4),
            ])
        ])
    ])
])

# Define the Sales dashboard layout
sales_layout = dbc.Container(fluid=True, children=[
    dbc.Row([
        dbc.Col([
            html.H2("Sales and Purchase Analysis"),
            html.Hr(),
            dbc.Row([
                dbc.Col([
                    html.Label("Item Name", className="dropdown-container"),
                    dcc.Dropdown(
                        id='item-name-filter',
                        options=[{'label': name, 'value': name} for name in sales_df['item_name'].unique()],
                        multi=True,
                        placeholder="Select Item(s)"
                    ),
                ], width=3),
                dbc.Col([
                    html.Label("Doctor Name", className="dropdown-container"),
                    dcc.Dropdown(
                        id='doctor-name-filter-sales',
                        options=[{'label': name, 'value': name} for name in sales_df['doctor_name'].unique()],
                        multi=True,
                        placeholder="Select Doctor(s)"
                    ),
                ], width=3),
                dbc.Col([
                    html.Label("Supplier Name", className="dropdown-container"),
                    dcc.Dropdown(
                        id='supplier-name-filter',
                        options=[{'label': name, 'value': name} for name in purchase_df['Supplier_name'].unique()],
                        multi=True,
                        placeholder="Select Supplier(s)"
                    ),
                ], width=3),
                dbc.Col([
                    html.Label("Sales Time", className="dropdown-container"),
                    dcc.Dropdown(
                        id='sales-time-filter',
                        options=[{'label': time, 'value': time} for time in sales_df['time'].unique()],
                        multi=True,
                        placeholder="Select Time(s)"
                    ),
                ], width=3),
            ]),
        ], width=12),
        dbc.Col([
            dbc.Row([
                dbc.Col(dcc.Graph(id='top-selling-items-chart', config={'displayModeBar': False}, className="graph-container"), width=6),
                dbc.Col(dcc.Graph(id='sales-time-chart', config={'displayModeBar': False}, className="graph-container"), width=4),

            ]),
            dbc.Row([
                dbc.Col(dcc.Graph(id='supplier-purchase-chart', config={'displayModeBar': False}, className="graph-container"), width=12),
            ]),
            dbc.Row([
                dbc.Col(dcc.Graph(id='monthly-sales-chart', config={'displayModeBar': False}, className="graph-container"), width=4),
                dbc.Col(dcc.Graph(id='doctor-sales-chart', config={'displayModeBar': False}, className="graph-container"), width=4),
            ]),

        ])
    ])
])

# Define the Lab dashboard layout
lab_layout = dbc.Container(fluid=True, children=[
    dbc.Row([
        dbc.Col([
            html.H2("Lab Analysis Dashboard"),
            html.Hr(),
            dbc.Row([
                dbc.Col([
                    html.Label("Test Name", className="dropdown-container"),
                    dcc.Dropdown(
                        id='test-name-filter',
                        options=[{'label': name, 'value': name} for name in lab_df['test_name'].unique()],
                        multi=True,
                        placeholder="Select Test Name(s)"
                    ),
                ], width=6),
                dbc.Col([
                    html.Label("Test Time", className="dropdown-container"),
                    dcc.Dropdown(
                        id='test-time-filter',
                        options=[{'label': time, 'value': time} for time in lab_df['time'].unique()],
                        multi=True,
                        placeholder="Select Test Time(s)"
                    ),
                ], width=6),
            ]),
        ], width=12),
        dbc.Col([
            dbc.Row([
                dbc.Col(dcc.Graph(id='test-traffic-chart', config={'displayModeBar': False}, className="graph-container"), width=4),
                dbc.Col(dcc.Graph(id='test-revenue-chart', config={'displayModeBar': False}, className="graph-container"), width=8),
            ]),
            dbc.Row([
                dbc.Col(dcc.Graph(id='charge-analysis-chart', config={'displayModeBar': False}, className="graph-container"), width=12),
            ])
        ])
    ])
])

# Update the URL and display the appropriate layout
@app.callback(Output('page-content', 'children'), [Input('url', 'pathname')])
def display_page(pathname):
    if pathname in ['/opd', '/']:
        return opd_layout
    elif pathname == '/sales':
        return sales_layout
    elif pathname == '/lab':
        return lab_layout
    else:
        return opd_layout  # Default to OPD layout if path is not recognized


# Callbacks for OPD Dashboard
@app.callback(
    [
        Output('visit-type-chart', 'figure'),
        Output('doctor-demand-chart', 'figure'),
        Output('registration-traffic-chart', 'figure'),
        Output('gender-analysis-chart', 'figure'),
        Output('age-analysis-chart', 'figure')
    ],
    [
        Input('doctor-name-filter', 'value'),
        Input('visit-type-filter', 'value'),
        Input('age-group-filter', 'value')
    ]
)
def update_opd_charts(doctor_names, visit_types, age_groups):
    filtered_df = opd_df.copy()

    if doctor_names:
        filtered_df = filtered_df[filtered_df['doctor_name'].isin(doctor_names)]
    if visit_types:
        filtered_df = filtered_df[filtered_df['visit_type'].isin(visit_types)]
    if age_groups:
        filtered_df = filtered_df[filtered_df['age'].isin(age_groups)]

    # Visit Type Analysis
    visit_type_fig = px.bar(filtered_df.groupby('visit_type').size().reset_index(name='count'), 
                            x='visit_type', y='count', color='visit_type', 
                            title='Visit Type Analysis', labels={'visit_type': 'Visit Type', 'count': 'Count of Visits'},
                            template='plotly_white')

    visit_type_fig.update_traces(marker_line_width=0.5, width=0.5)

    doctor_demand_fig = px.bar(filtered_df.groupby('doctor_name').size().reset_index(name='count'), 
                        y='doctor_name', x='count', color='doctor_name', orientation='h',
                        title='Doctor Demand Analysis', labels={'count': 'Count of Patients', 'doctor_name': 'Doctor Name'},
                        template='plotly_white')
    doctor_demand_fig.update_layout(bargap=0.4) 
    # Hide legend in Doctor Demand Analysis
    doctor_demand_fig.update_traces(showlegend=False)

    # Group the data by 'time' and count the unique 'patient_id' for each time period
    time_counts = filtered_df.groupby('time')['patient_id'].count().reset_index()
    time_counts.columns = ['time', 'count']

    # Create the pie chart
    reg_traffic_fig = px.pie(time_counts, names='time', values='count',
                             title='Registration Traffic Analysis', labels={'time': 'Time of Visit', 'count': 'Count of Patients'},
                             template='plotly_white')
    # Group the data by gender and count the occurrences
    gender_counts = filtered_df['gender'].value_counts().reset_index()
    gender_counts.columns = ['gender', 'count']

    # Create the pie chart
    gender_fig = px.pie(gender_counts, names='gender', values='count', hole=0.3,
                        title='Gender Analysis', labels={'gender': 'Gender', 'count': 'Count of Patients'},
                        template='plotly_white')
    # Age Analysis
    age_fig = px.histogram(filtered_df, x='age', color='age', barmode='stack',
                            title='Age Analysis', labels={'age': 'Age Group', 'count': 'Count of Patients'},
                            template='plotly_white')

    return visit_type_fig, doctor_demand_fig, reg_traffic_fig, gender_fig, age_fig

# Callbacks for Sales Dashboard
@app.callback(
    [
        Output('top-selling-items-chart', 'figure'),
        Output('sales-time-chart', 'figure'),
        Output('monthly-sales-chart', 'figure'),
        Output('doctor-sales-chart', 'figure'),
        Output('supplier-purchase-chart', 'figure')
    ],
    [
        Input('item-name-filter', 'value'),
        Input('doctor-name-filter-sales', 'value'),
        Input('supplier-name-filter', 'value'),
        Input('sales-time-filter', 'value')
    ]
)
def update_sales_charts(item_names, doctor_names, supplier_names, sales_times):
    sales_filtered_df = sales_df.copy()
    purchase_filtered_df = purchase_df.copy()

    if item_names:
        sales_filtered_df = sales_filtered_df[sales_filtered_df['item_name'].isin(item_names)]
    if doctor_names:
        sales_filtered_df = sales_filtered_df[sales_filtered_df['doctor_name'].isin(doctor_names)]
    if supplier_names:
        purchase_filtered_df = purchase_filtered_df[purchase_filtered_df['Supplier_name'].isin(supplier_names)]
    if sales_times:
        sales_filtered_df = sales_filtered_df[sales_filtered_df['time'].isin(sales_times)]

    top_selling_items_df = sales_filtered_df['item_name'].value_counts().nlargest(5).reset_index()
    top_selling_items_df.columns = ['item_name', 'count']
    top_selling_items_fig = px.bar(top_selling_items_df, y='item_name', x='count', color='item_name', orientation='h',
                                   title='Top Selling Items', labels={'count': 'Count of Sales', 'item_name': 'Item Name'},
                                   template='plotly_white')
    top_selling_items_fig.update_layout(bargap=0.4)
    top_selling_items_fig.update_traces(showlegend=False)

    # Group the data by 'time' and count the occurrences of 'sale_id' for each time period
    time_sales_counts = sales_filtered_df.groupby('time')['sale_id'].count().reset_index()
    time_sales_counts.columns = ['time', 'count']

    # Create the pie chart
    sales_time_fig = px.pie(time_sales_counts, names='time', values='count',
                            title='Sales Time Analysis', labels={'time': 'Time of Sale', 'count': 'Count of Sales'},
                            template='plotly_white')

    # Show the legend with time labels and display percentage + label inside the pie chart
    sales_time_fig.update_traces(textposition='inside', textinfo='percent+label')

    monthly_sales_df = sales_filtered_df.groupby('month').size().reset_index(name='count')
    monthly_sales_fig = px.bar(monthly_sales_df, x='month', y='count', color='month',
                               title='Monthly Sales Analysis', labels={'month': 'Month', 'count': 'Count of Sales'},
                               template='plotly_white')

    doctor_sales_df = sales_filtered_df.groupby('doctor_name').size().reset_index(name='count')
    doctor_sales_fig = px.bar(doctor_sales_df, y='doctor_name', x='count', color='doctor_name', orientation='h',
                              title='Doctor Sales Analysis', labels={'count': 'Count of Sales', 'doctor_name': 'Doctor Name'},
                              template='plotly_white')
    doctor_sales_fig.update_layout(bargap=0.4)
    doctor_sales_fig.update_traces(showlegend=False)

    supplier_purchase_df = purchase_filtered_df.groupby('Supplier_name')['Amount_total'].sum().nlargest(10).reset_index()
    supplier_purchase_fig = px.bar(supplier_purchase_df, y='Supplier_name', x='Amount_total', color='Supplier_name', orientation='h',
                                   title='Supplier Purchase Analysis', labels={'Amount_total': 'Total Amount', 'Supplier_name': 'Supplier Name'},
                                   template='plotly_white')
    supplier_purchase_fig.update_layout(bargap=0.4)
    supplier_purchase_fig.update_traces(showlegend=False)

    return top_selling_items_fig, sales_time_fig, monthly_sales_fig, doctor_sales_fig, supplier_purchase_fig

# Callbacks for Lab Dashboard
@app.callback(
    [
        Output('test-traffic-chart', 'figure'),
        Output('test-revenue-chart', 'figure'),
        Output('charge-analysis-chart', 'figure')
    ],
    [
        Input('test-name-filter', 'value'),
        Input('test-time-filter', 'value')
    ]
)
def update_lab_charts(test_names, test_times):
    filtered_df = lab_df.copy()

    if test_names:
        filtered_df = filtered_df[filtered_df['test_name'].isin(test_names)]
    if test_times:
        filtered_df = filtered_df[filtered_df['time'].isin(test_times)]
    # Aggregate data for pie chart
    traffic_data = filtered_df.groupby('time')['test_id'].count().reset_index(name='count')

     # Create the pie chart for Test Traffic Analysis
    test_traffic_fig = px.pie(traffic_data, names='time', values='count',
                              title='Test Traffic Analysis', labels={'time': 'Time', 'count': 'Count of test_id'},
                              template='plotly_white')
    test_revenue_fig = px.bar(filtered_df.groupby('test_name').size().nlargest(10).reset_index(name='count'), 
                              y='test_name', x='count', color='test_name', orientation='h',
                              title='Test Revenue Analysis', labels={'count': 'Count of Tests', 'test_name': 'Test Name'},
                              template='plotly_white')
    test_revenue_fig.update_layout(bargap=0.4)
    test_revenue_fig.update_traces(showlegend=False)

    charge_analysis_fig = px.bar(filtered_df.groupby('test_name')['charge'].sum().nlargest(10).reset_index(name='total_charge'), 
                                 y='test_name', x='total_charge', color='test_name', orientation='h',
                                 title='Charge Analysis', labels={'total_charge': 'Total Charge', 'test_name': 'Test Name'},
                                 template='plotly_white')
    charge_analysis_fig.update_layout(bargap=0.4)
    charge_analysis_fig.update_traces(showlegend=False)

    return test_traffic_fig, test_revenue_fig, charge_analysis_fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

C:\Users\adnan\AppData\Local\Temp\ipykernel_13864\1266932207.py:29: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\adnan\AppData\Local\Temp\ipykernel_13864\1266932207.py:30: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\adnan\AppData\Local\Temp\ipykernel_13864\1266932207.py:31: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\adnan\AppData\Local\Temp\ipykernel_13864\1266932207.py:32: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 